# Import Packages and TFRS

In [2]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

     |████████████████████████████████| 3.6MB 2.8MB/s 


In [4]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs
from typing import Dict, Text

## Read the data

In [5]:
# Ratings data.
ratings = tfds.load('movielens/100k-ratings', split="train")

Shuffling and writing examples to /root/tensorflow_datasets/movielens/100k-ratings/0.1.0.incomplete5P7S30/movielens-train.tfrecord


Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-ratings/0.1.0. Subsequent calls will reuse this data.


In [6]:
# Features of all the available movies.
movies = tfds.load('movielens/100k-movies', split="train")

Shuffling and writing examples to /root/tensorflow_datasets/movielens/100k-movies/0.1.0.incomplete717NET/movielens-train.tfrecord


Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-movies/0.1.0. Subsequent calls will reuse this data.


In [7]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})

In [8]:
movies = movies.map(lambda x: x["movie_title"])

In [9]:
user_ids_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

In [10]:
movie_titles_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

# Define Model

In [11]:
class MovieLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.movie_model = movie_model

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

    user_embeddings = self.user_model(features["user_id"])
    movie_embeddings = self.movie_model(features["movie_title"])

    return self.task(user_embeddings, movie_embeddings)

In [12]:
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 64)
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), 64)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)
  )
)

# Fit and evaluate it

In [16]:
# Create a retrieval model.
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 3 epochs.
model.fit(ratings.batch(4096), epochs=3)

# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index(movies.batch(100).map(model.movie_model), movies)


Epoch 1/3


25/25 [==============================] - 26s 1s/step - factorized_top_k/top_1_categorical_accuracy: 2.9000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0043 - factorized_top_k/top_10_categorical_accuracy: 0.0102 - factorized_top_k/top_50_categorical_accuracy: 0.0732 - factorized_top_k/top_100_categorical_accuracy: 0.1558 - loss: 33206.2875 - regularization_loss: 0.0000e+00 - total_loss: 33206.2875
Epoch 2/3
25/25 [==============================] - 26s 1s/step - factorized_top_k/top_1_categorical_accuracy: 6.1000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0075 - factorized_top_k/top_10_categorical_accuracy: 0.0202 - factorized_top_k/top_50_categorical_accuracy: 0.1340 - factorized_top_k/top_100_categorical_accuracy: 0.2552 - loss: 30600.0482 - regularization_loss: 0.0000e+00 - total_loss: 30600.0482
Epoch 3/3
25/25 [==============================] - 26s 1s/step - factorized_top_k/top_1_categorical_accuracy: 5.3000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.01

In [17]:
# Get some recommendations.
_, titles = index(np.array(["42"]))
print(f"Top 3 recommendations for user 42: {titles[0, :3]}")

Top 3 recommendations for user 42: [b'Rent-a-Kid (1995)' b'House Arrest (1996)'
 b'Winnie the Pooh and the Blustery Day (1968)']
